In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
## Model1: Simple CNN

In [ ]:
## Model2: ResNet with transfer learning

In [ ]:
## Model3: HOG classification